In [6]:
import cv2 as cv

frame = cv.imread('../data/kobe.jpg')
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

ret, thresh = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)

cv.imshow('thresh', thresh)
cv.waitKey(0)
cv.destroyAllWindows()
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

for contour in contours:
    hull = cv.convexHull(contour)

    cv.drawContours(frame, [hull], -1, (0, 255, 0), 2)

cv.imshow('Convex Hull', frame)
cv.waitKey(0)
cv.destroyAllWindows()


In [4]:
import cv2 as cv

cam = cv.VideoCapture(0)

while True:

    ret, frame = cam.read()

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (5, 5), 0)

    _, thresh = cv.threshold(blurred, 60, 255, cv.THRESH_BINARY_INV)

    contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    contour = max(contours, key=cv.contourArea)

    hull = cv.convexHull(contour)
    hull_indices = cv.convexHull(contour, returnPoints=False)
    defects = cv.convexityDefects(contour, hull_indices)

    cv.drawContours(frame, [contour], -1, (0, 255, 0), 2)
    cv.drawContours(frame, [hull], -1, (255, 0, 0), 2)

    if defects is not None:
        for i in range(defects.shape[0]):
            start_idx, end_idx, far_idx, _ = defects[i, 0]
            start = tuple(contour[start_idx][0])
            end = tuple(contour[end_idx][0])
            far = tuple(contour[far_idx][0])
            cv.circle(frame, far, 5, (0, 0, 255), -1)

    cv.imshow('Hand Gesture', frame)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

cam.release()
cv.destroyAllWindows()